In [1]:
from __future__ import absolute_import, division, print_function

import os
import tensorflow as tf
from tensorflow_transform.tf_metadata import dataset_metadata, metadata_io,dataset_schema 
from tensorflow_transform.beam.tft_beam_io import transform_fn_io
tf.enable_eager_execution()

In [2]:
os.getcwd()

'/Users/Niklas/Documents/Named_entity_recognition/model/notebooks'

# HAVE TO RE RUN ALL! AND CLEAR FOLDERS

https://guillaumegenthial.github.io/sequence-tagging-with-tensorflow.html

In [3]:
transformed_metadata = metadata_io.read_metadata(
    os.path.join(
      "gs://named_entity_recognition/beam/", transform_fn_io.TRANSFORMED_METADATA_DIR
      )
    )
transformed_feature_spec = transformed_metadata.schema.as_feature_spec()

In [4]:
transformed_feature_spec

{u'chars': FixedLenFeature(shape=[30, 10], dtype=tf.int64, default_value=None),
 u'chars_in_word': FixedLenFeature(shape=[30], dtype=tf.int64, default_value=None),
 u'id': FixedLenFeature(shape=[], dtype=tf.float32, default_value=None),
 u'label': FixedLenFeature(shape=[], dtype=tf.string, default_value=None),
 u'label_length': FixedLenFeature(shape=[], dtype=tf.int64, default_value=None),
 u'labels': FixedLenFeature(shape=[30], dtype=tf.int64, default_value=None),
 u'sentence_length': FixedLenFeature(shape=[], dtype=tf.int64, default_value=None),
 u'text': FixedLenFeature(shape=[], dtype=tf.string, default_value=None),
 u'word_representation': VarLenFeature(dtype=tf.int64)}

In [5]:
def _read_from_tfrecord(example_proto):
    features = tf.parse_example([example_proto], features=transformed_feature_spec)

    return features

In [139]:
output_list=[]

In [140]:
ds = tf.data.TFRecordDataset("gs://named_entity_recognition/beam/TEST-00000-of-00001").map(_read_from_tfrecord).shuffle(10).batch(2).make_one_shot_iterator()
for x in ds:
    output_list.append(x)


In [143]:
for x in output_list:
    print(x["labels"])

tf.Tensor(
[[[2 8 8 0 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8]]

 [[2 8 8 0 1 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8]]], shape=(2, 1, 30), dtype=int64)
tf.Tensor(
[[[2 8 8 0 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8]]

 [[2 8 8 0 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8]]], shape=(2, 1, 30), dtype=int64)
tf.Tensor(
[[[2 8 8 0 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8]]

 [[2 8 8 0 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8]]], shape=(2, 1, 30), dtype=int64)
tf.Tensor(
[[[2 8 8 0 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8]]

 [[2 8 8 0 1 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8]]], shape=(2, 1, 30), dtype=int64)
tf.Tensor([[[2 8 8 0 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8]]], shape=(1, 1, 30), dtype=int64)


In [44]:
from string import ascii_lowercase
MAPPING = {a:index for index,a in enumerate(ascii_lowercase + ascii_lowercase.upper())}

In [45]:
x.keys()

[u'word_representation',
 u'text',
 u'chars',
 u'labels',
 u'label',
 u'label_length',
 u'sentence_length',
 u'id',
 u'chars_in_word']

In [46]:
print(x['labels'])
print(x['sentence_length'])

tf.Tensor(
[[[2 8 8 0 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8]]

 [[2 8 8 0 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8]]], shape=(2, 1, 30), dtype=int64)
tf.Tensor(
[[4]
 [4]], shape=(2, 1), dtype=int64)


In [84]:
SENTENCE_LEN=30
WORD_LEN=10
BATCH_SIZE=2
CHAR_SIZE=54
EMBED_DIM_CHAR=int(CHAR_SIZE**(1/4))*8
DEFAULT_WORD_VALUE =0
VOCAB_SIZE=1e4
EMBED_DIM_WORD=int(VOCAB_SIZE**(1/4))*8
input_char = tf.reshape(x["chars"],[BATCH_SIZE,SENTENCE_LEN,WORD_LEN])
char_embedding = tf.contrib.layers.embed_sequence(
    input_char, vocab_size=CHAR_SIZE, embed_dim=EMBED_DIM_CHAR)

In [93]:
# No padding needed now :) 
input_word = tf.sparse_to_dense(x["word_representation"].indices,[BATCH_SIZE,1,SENTENCE_LEN],
                           x["word_representation"].values,default_value=DEFAULT_WORD_VALUE
                          )
input_word = tf.reshape(input_word,[BATCH_SIZE,SENTENCE_LEN])
word_embedding = tf.contrib.layers.embed_sequence(
    input_word, vocab_size=VOCAB_SIZE, embed_dim=EMBED_DIM_WORD)

In [94]:
x["labels"]

<tf.Tensor: id=821, shape=(2, 1, 30), dtype=int64, numpy=
array([[[2, 8, 8, 0, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8,
         8, 8, 8, 8, 8, 8, 8, 8, 8]],

       [[2, 8, 8, 0, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8,
         8, 8, 8, 8, 8, 8, 8, 8, 8]]])>

In [95]:
x["labels"]

<tf.Tensor: id=821, shape=(2, 1, 30), dtype=int64, numpy=
array([[[2, 8, 8, 0, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8,
         8, 8, 8, 8, 8, 8, 8, 8, 8]],

       [[2, 8, 8, 0, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8,
         8, 8, 8, 8, 8, 8, 8, 8, 8]]])>

In [96]:
input_word

<tf.Tensor: id=1985, shape=(2, 30), dtype=int64, numpy=
array([[14,  0,  1, 10,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
       [17,  0,  1, 20,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0]])>

In [97]:
# Print the shapes 
print(tf.shape(input_word))
print(tf.shape(word_embedding))
print(tf.shape(char_embedding))

tf.Tensor([ 2 30], shape=(2,), dtype=int32)
tf.Tensor([ 2 30 80], shape=(3,), dtype=int32)
tf.Tensor([ 2 30 10 16], shape=(4,), dtype=int32)


In [98]:
s = tf.shape(char_embedding)
char_embeddings = tf.reshape(char_embedding, shape=[-1, s[-2], s[-1]])
word_lengths = tf.reshape(x['chars_in_word'],[-1])

In [99]:
CHAR_HIDDEN_SIZE=10
# 3. bi lstm on chars
cell_fw = tf.contrib.rnn.LSTMCell(CHAR_HIDDEN_SIZE, state_is_tuple=True)
cell_bw = tf.contrib.rnn.LSTMCell(CHAR_HIDDEN_SIZE, state_is_tuple=True)

_, ((_, output_fw), (_, output_bw)) = tf.nn.bidirectional_dynamic_rnn(cell_fw,
    cell_bw, char_embeddings, sequence_length=word_lengths,
    dtype=tf.float32)
#shape = (batch x sentence, 2 x char_hidden_size)
output = tf.concat([output_fw, output_bw], axis=-1)

# shape = (batch, sentence, 2 x char_hidden_size)
char_rep = tf.reshape(output, shape=[-1, s[1], 2*CHAR_HIDDEN_SIZE])

# shape = (batch, sentence, 2 x char_hidden_size + word_vector_size)
word_embeddings = tf.concat([word_embedding, char_rep], axis=-1)

In [100]:
word_lengths

<tf.Tensor: id=2022, shape=(60,), dtype=int64, numpy=
array([5, 5, 2, 5, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 7, 5, 2, 6, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])>

In [101]:
dim_words = tf.shape(char_embeddings)[1]
dim_chars = tf.shape(char_embeddings)[2]
flat = tf.reshape(char_embeddings, [-1, dim_words,EMBED_DIM_CHAR])


In [102]:
tf.shape(char_embeddings)
t = tf.transpose(char_embeddings, perm=[1, 0, 2])
print(t)

tf.Tensor(
[[[ 0.11493748 -0.18082732 -0.08957429 ... -0.2770458   0.02863061
   -0.2883652 ]
  [-0.03224564 -0.24759862 -0.0166758  ... -0.0525489   0.00500345
   -0.240506  ]
  [-0.16266385 -0.26379517  0.27315307 ... -0.05415455 -0.28551635
   -0.14136389]
  ...
  [ 0.02873635 -0.24445572 -0.21755669 ...  0.13667998  0.16544306
    0.10515153]
  [ 0.02873635 -0.24445572 -0.21755669 ...  0.13667998  0.16544306
    0.10515153]
  [ 0.02873635 -0.24445572 -0.21755669 ...  0.13667998  0.16544306
    0.10515153]]

 [[ 0.14575443  0.1375423  -0.28184757 ...  0.28168172 -0.23467132
    0.03394204]
  [-0.16266385 -0.26379517  0.27315307 ... -0.05415455 -0.28551635
   -0.14136389]
  [ 0.16457126 -0.1254763  -0.28730524 ...  0.06018466 -0.17130196
   -0.03348517]
  ...
  [ 0.02873635 -0.24445572 -0.21755669 ...  0.13667998  0.16544306
    0.10515153]
  [ 0.02873635 -0.24445572 -0.21755669 ...  0.13667998  0.16544306
    0.10515153]
  [ 0.02873635 -0.24445572 -0.21755669 ...  0.13667998  0.1654

In [103]:
print(tf.shape(char_embeddings))
print(dim_chars)
print(dim_words)
print(EMBED_DIM_CHAR)

tf.Tensor([60 10 16], shape=(3,), dtype=int32)
tf.Tensor(16, shape=(), dtype=int32)
tf.Tensor(10, shape=(), dtype=int32)
16


In [106]:
# WORD LSTM:S

HIDDEN_SIZE=20
cell_fw = tf.contrib.rnn.LSTMCell(HIDDEN_SIZE)
cell_bw = tf.contrib.rnn.LSTMCell(HIDDEN_SIZE)
sequence_length= tf.reshape(x['sentence_length'],[BATCH_SIZE])
(output_fw, output_bw), _ = tf.nn.bidirectional_dynamic_rnn(cell_fw,
    cell_bw, word_embeddings, sequence_length=sequence_length,
    dtype=tf.float32)
context_rep = tf.concat([output_fw, output_bw], axis=-1)

In [108]:
tf.shape(context_rep)

<tf.Tensor: id=5381, shape=(3,), dtype=int32, numpy=array([ 2, 30, 40], dtype=int32)>

In [118]:
CLASSES_OUTPUT=9

ntime_steps = tf.shape(context_rep)[1]
context_rep_flat = tf.reshape(context_rep, [-1, 2*HIDDEN_SIZE])
preds = tf.layers.dense(inputs = context_rep_flat,units=CLASSES_OUTPUT)
logits = tf.reshape(preds, [-1, ntime_steps, CLASSES_OUTPUT])


In [119]:
crf_params = tf.get_variable("crf", [CLASSES_OUTPUT, CLASSES_OUTPUT], dtype=tf.float32)
pred_ids, _ = tf.contrib.crf.crf_decode(logits, crf_params, sequence_length)

In [147]:
sequence_length

<tf.Tensor: id=2971, shape=(2,), dtype=int64, numpy=array([4, 4])>

In [148]:
sequence_length

<tf.Tensor: id=2971, shape=(2,), dtype=int64, numpy=array([4, 4])>

In [151]:
weights = tf.sequence_mask(sequence_length,maxlen=20)
weights
#tf.metrics.accuracy(labels, pred_ids, weights)

<tf.Tensor: id=13051, shape=(2, 20), dtype=bool, numpy=
array([[ True,  True,  True,  True, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False,
        False, False],
       [ True,  True,  True,  True, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False,
        False, False]])>

In [146]:
pred_ids

<tf.Tensor: id=10353, shape=(2, 30), dtype=int32, numpy=
array([[3, 0, 0, 5, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0],
       [3, 0, 0, 5, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0]], dtype=int32)>

In [133]:

labels = tf.reshape(x['labels'],[BATCH_SIZE,SENTENCE_LEN])
labels

<tf.Tensor: id=11559, shape=(2, 30), dtype=int64, numpy=
array([[2, 8, 8, 0, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8,
        8, 8, 8, 8, 8, 8, 8, 8],
       [2, 8, 8, 0, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8,
        8, 8, 8, 8, 8, 8, 8, 8]])>

In [134]:
logits

<tf.Tensor: id=8745, shape=(2, 30, 9), dtype=float32, numpy=
array([[[-0.01310119, -0.00056921,  0.00136848,  0.01918247,
          0.01797333,  0.00120888, -0.00618167,  0.00519746,
          0.0076556 ],
        [-0.01320264,  0.0033567 ,  0.0090897 ,  0.02245667,
          0.00786415, -0.00058932, -0.0046156 ,  0.00271039,
          0.02036974],
        [-0.01781182,  0.0007734 ,  0.01562909,  0.01014564,
          0.00997417, -0.00045097, -0.00981556,  0.00236148,
          0.00493213],
        [-0.01214561, -0.00469819,  0.00731217,  0.01789845,
          0.00397268,  0.00260659, -0.01979513, -0.0002396 ,
          0.00149634],
        [ 0.        ,  0.        ,  0.        ,  0.        ,
          0.        ,  0.        ,  0.        ,  0.        ,
          0.        ],
        [ 0.        ,  0.        ,  0.        ,  0.        ,
          0.        ,  0.        ,  0.        ,  0.        ,
          0.        ],
        [ 0.        ,  0.        ,  0.        ,  0.        ,
        

In [135]:
log_likelihood, _ = tf.contrib.crf.crf_log_likelihood(
        logits, labels, sequence_length, crf_params)
loss = tf.reduce_mean(-log_likelihood)

In [136]:
loss

<tf.Tensor: id=12710, shape=(), dtype=float32, numpy=9.030251>

In [ ]:
LOOK_UP_TABLE="FILE_NAME_TO_LOOP_UP_VOCAB"

if mode == tf.estimator.ModeKeys.PREDICT:
    # Predictions
    # GET THE WORD FORM THE ID, LINK THIS WITH THE BEAM TRANSFORM VOCAB
    reverse_vocab_tags = tf.contrib.lookup.index_to_string_table_from_file(
        LOOK_UP_TABLE)
    pred_strings = reverse_vocab_tags.lookup(tf.to_int64(pred_ids))
    predictions = {
        'pred_ids': pred_ids,
        'tags': pred_strings
    }
    return tf.estimator.EstimatorSpec(mode, predictions=predictions)
else:
    # Loss
    tags = labes
    log_likelihood, _ = tf.contrib.crf.crf_log_likelihood(
        logits, tags, nwords, crf_params)
    loss = tf.reduce_mean(-log_likelihood)

    # Metrics
    # AWESOME THIS IS GREAT TO USE THIS MASK HERE
    # WILL SOLVE A LOT OF PROBLEMS, OTHERWISE
    # GOOD LEARNING AS WELL
    weights = tf.sequence_mask(nwords)
    metrics = {
        'acc': tf.metrics.accuracy(tags, pred_ids, weights),
        'precision': precision(tags, pred_ids, num_tags, indices, weights),
        'recall': recall(tags, pred_ids, num_tags, indices, weights),
        'f1': f1(tags, pred_ids, num_tags, indices, weights),
    }
    # ADD THEM ALL TO THE GRAPH
    for metric_name, op in metrics.items():
        tf.summary.scalar(metric_name, op[1])
    
    # WHAT WILL HAPPEN IF WE HAVE EVAL MODE
    if mode == tf.estimator.ModeKeys.EVAL:
        return tf.estimator.EstimatorSpec(
            mode, loss=loss, eval_metric_ops=metrics)

    # WHAT WILL HAPPEN IF WE HAVE TRAIN? 
    elif mode == tf.estimator.ModeKeys.TRAIN:
        train_op = tf.train.AdamOptimizer().minimize(
            loss, global_step=tf.train.get_or_create_global_step())
        return tf.estimator.EstimatorSpec(
            mode, loss=loss, train_op=train_op)

In [145]:
NUM_CLASSES = 9
NUM_CLASSES = tf.constant(NUM_CLASSES)
NUM_CLASSES

<tf.Tensor: id=13026, shape=(), dtype=int32, numpy=9>